In [5]:
import sys
import os


chemin_dossier = os.path.abspath('../Project_scripts')
if chemin_dossier not in sys.path:
    sys.path.append(chemin_dossier)



In [6]:
from scryfall_api import *
from manage_data import *
import time

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def scrape_edhrec():
    url = "https://edhrec.com/top/month"
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get(url)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    top_cards = {}
    for i, card_div in enumerate(soup.select('.Card_name__Mpa7S'), 1):
        card_name = card_div.get_text(strip=True)
        top_cards[card_name] = i

    driver.quit()
    return top_cards  # dict: {name: rank}

def scrape_mtggoldfish():
    urls = {
        "pauper": "https://www.mtggoldfish.com/format-staples/pauper/full/all",
        "standard": "https://www.mtggoldfish.com/format-staples/standard/full/all",
        "modern": "https://www.mtggoldfish.com/format-staples/modern/full/all",
        "vintage": "https://www.mtggoldfish.com/format-staples/vintage/full/all"
    }

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    card_rankings = {}

    for format_name, url in urls.items():
        driver.get(url)
        time.sleep(5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        for i, card_div in enumerate(soup.select('.card_name'), 1):
            card_name = card_div.get_text(strip=True)
            if card_name not in card_rankings:
                card_rankings[card_name] = {}
            card_rankings[card_name][format_name] = i

    driver.quit()
    return card_rankings  # dict[card_name] = {format: rank}

def get_all_popular_cards():
    edhrec_data = scrape_edhrec()
    goldfish_data = scrape_mtggoldfish()

    all_card_names = set(edhrec_data.keys()) | set(goldfish_data.keys())

    result = []
    for name in sorted(all_card_names):
        entry = [
            name,
            edhrec_data.get(name),
            goldfish_data.get(name, {}).get("pauper"),
            goldfish_data.get(name, {}).get("standard"),
            goldfish_data.get(name, {}).get("modern"),
            goldfish_data.get(name, {}).get("vintage"),
        ]
        result.append(entry)

    return result



In [8]:
popularity_list = get_all_popular_cards()

In [9]:
print(popularity_list[0])

['Abrade', None, None, 2, None, None]


In [10]:
query_reset = """
       DELETE FROM popularity
    """

In [11]:
query_popularity ="""
        INSERT INTO popularity (
             name,commander,pauper,standard,modern,vintage)
        VALUES (%s, %s, %s, %s, %s, %s)
    """

In [19]:
conn, cur = connect_database()

In [20]:
cur.execute(query_reset)

In [15]:
cur.executemany(query_popularity, popularity_list)

In [21]:
conn.commit()

In [4]:
BACKUP
def get_all_popular_cards():
    edhrec_data = scrape_edhrec()
    goldfish_data = scrape_mtggoldfish()

    all_card_names = set(edhrec_data.keys()) | set(goldfish_data.keys())

    result = []
    for name in sorted(all_card_names):
        entry = {
            "name": name,
            "commander": edhrec_data.get(name),
            "pauper": goldfish_data.get(name, {}).get("pauper"),
            "standard": goldfish_data.get(name, {}).get("standard"),
            "modern": goldfish_data.get(name, {}).get("modern"),
            "vintage": goldfish_data.get(name, {}).get("vintage"),
        }
        result.append(entry)

    return result

NameError: name 'BACKUP' is not defined